In [14]:
import os
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import exc
from io import StringIO
import re
import sys
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import json

In [15]:
# Adicione aqui sua Consumer Key
consumer_key = "NrpJxCRklgbJjW2N4IgNEpRKW"

# Adicione aqui sua Consumer Secret 
consumer_secret = "lQ3n7joXCSdVqjWAT0LsA0RjQCZpbqL7wjZxncCJW3afw5og3s"

# Adicione aqui seu Access Token
access_token = "1083158543967158272-YeGirzCjTtECUz2s5nUKQzy7kRTj9A"

# Adicione aqui seu Access Token Secret
access_token_secret = "KHo7aE9zAIzY4JcM7cX9jiyvGelmbK0sBCezgdY9scXXL"

# Criando as chaves de autenticação
auth = OAuthHandler(consumer_key, consumer_secret)

auth.set_access_token(access_token, access_token_secret)

In [16]:
# Criando uma classe para capturar os stream de dados do Twitter e 
# armazenar no MongoDB
class MyListener(StreamListener):
    def on_data(self, dados):
        tweet = json.loads(dados)
        created_at = tweet["created_at"]
        id_str = tweet["id_str"]
        user = tweet["user"]
        text = tweet["text"]
        obj = {"created_at":created_at,"id_str":id_str,"user":user,"text":text,}
        tweetind = col.insert_one(obj).inserted_id
        print (obj)
        return True

In [17]:
# Criando o objeto mylistener
mylistener = MyListener()

In [18]:
# Criando o objeto mystream
mystream = Stream(auth, listener = mylistener)

In [19]:
# Preparando a Conexão com o Postgres

In [20]:
engine = create_engine('postgresql+psycopg2://{}:{}@{}:{}/{}'.format(
    'postgres',
    'Ghm2020!',
    'ghmpsql.chlnyxyoysu0.us-east-2.rds.amazonaws.com',
    '5432' ,
    'postgres'
))

In [21]:
%env SCHEMA = bot_stage

env: SCHEMA=bot_stage


In [22]:
# Criando uma lista de palavras chave para buscar nos Tweets
keywords = ['Boticário', 'Boticario']

In [24]:
mystream.filter(track=keywords)#.length(50)

In [26]:
mystream

In [ ]:
# Coletando os Tweets

In [27]:
def load_to_db(mystream):
    df.to_sql('tweets'.format(tabela),
          con=engine,
          schema=os.getenv('SCHEMA'),
          if_exists='replace',
          index=False,
          chunksize=10000,
          method='multi'
         )

In [ ]:
mystream.disconnect()

In [31]:
# General:
import tweepy           # To consume Twitter's API
import pandas as pd     # To handle data
import numpy as np      # For number computing

# For plotting and visualization:
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [32]:
# Twitter App access keys for @user

# Consume:
CONSUMER_KEY    = "NrpJxCRklgbJjW2N4IgNEpRKW"
CONSUMER_SECRET = "lQ3n7joXCSdVqjWAT0LsA0RjQCZpbqL7wjZxncCJW3afw5og3s"

# Access:
ACCESS_TOKEN  = "1083158543967158272-YeGirzCjTtECUz2s5nUKQzy7kRTj9A"
ACCESS_SECRET = "KHo7aE9zAIzY4JcM7cX9jiyvGelmbK0sBCezgdY9scXXL"

In [34]:
# We import our access keys:
from credentials import *    # This will allow us to use the keys as variables

# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth)
    return api

In [37]:
# We create an extractor object:
extractor = twitter_setup()

# We create a tweet list as follows:
tweets = extractor.user_timeline(screen_name="realDonaldTrump", count=200)
print("Number of tweets extracted: {}.\n".format(len(tweets)))

# We print the most recent 5 tweets:
print("5 recent tweets:\n")
for tweet in tweets[:5]:
    print(tweet.text)
    print()

TweepError: [{'code': 89, 'message': 'Invalid or expired token.'}]

In [ ]:
# We create a pandas dataframe as follows:
data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data.head(10))

In [ ]:
# Internal methods of a single tweet object:
print(dir(tweets[0]))

In [ ]:
# We print info from the first tweet:
print(tweets[0].id)
print(tweets[0].created_at)
print(tweets[0].source)
print(tweets[0].favorite_count)
print(tweets[0].retweet_count)
print(tweets[0].geo)
print(tweets[0].coordinates)
print(tweets[0].entities)

In [ ]:
# We add relevant data:
data['len']  = np.array([len(tweet.text) for tweet in tweets])
data['ID']   = np.array([tweet.id for tweet in tweets])
data['Date'] = np.array([tweet.created_at for tweet in tweets])
data['Source'] = np.array([tweet.source for tweet in tweets])
data['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
data['RTs']    = np.array([tweet.retweet_count for tweet in tweets])

In [ ]:
# Display of first 10 elements from dataframe:
display(data.head(10))